Notebook to perform tests of runtime type checking in MicroPython.


In [37]:
# %load_ext micropython_magic

In [70]:
%run ./deploy.ipynb

Install __future__
Installing __future__ (latest) from https://micropython.org/pi/v2 to /lib
Installing: /lib/__future__.mpy
Done
d:\mypython\!-stubtestprojects\rt_typing\mp
d:\mypython\!-stubtestprojects\rt_typing
cp mp/typing.mpy :/lib/typing.mpy
cp mp/typing_extensions.mpy :/lib/typing_extensions.mpy
cp mp/abc.mpy :/lib/abc.mpy
mkdir :/lib/collections
Traceback (most recent call last):

  File "<stdin>", line 2, in <module>

OSError: [Errno 17] EEXIST


cp mp/collections/abc.mpy :/lib/collections/abc.mpy
rm :/collections
ls :
           0 lib/
ls :/lib
         202 __future__.mpy
          43 abc.mpy
           0 collections/
         321 typing.mpy
          66 typing_extensions.mpy
ls :/lib/collections
          55 abc.mpy
True


In [71]:
%mpy import sys;print(f"Testing on {sys.implementation.name} {sys.platform}")

['Testing on micropython rp2']

In [40]:
# %%micropython --reset
import __future__
import typing
import typing_extensions
import abc
from collections import abc

True


In [41]:
# %%micropython --reset

from typing import Protocol


class Adder(Protocol):
    def add(self, x, y): ...


class IntAdder:
    def add(self, x, y):
        return x + y


class FloatAdder:
    def add(self, x, y):
        return x + y


def add(adder: Adder) -> None:
    print(adder.add(2, 3))


add(IntAdder())
add(FloatAdder())

True
5
5


In [42]:
# %%micropython

from typing import List, Protocol

l: List[int] = [1, 2, 3]


class Speak(Protocol):
    def speak(self): ...


class Parrot:
    def speak(self) -> None:
        print("Polly wants a cracker")


def say_something(speaker: Speak) -> None:
    speaker.speak()


polly = Parrot()

print(say_something(polly))

Polly wants a cracker
None


In [43]:
# %%micropython --reset
from typing import TYPE_CHECKING, __ignore, trace

if TYPE_CHECKING:
    from abc import ABC, abstractmethod
    from math import pi


class ABC:
    """A class to ignore type hints in code."""

    @trace
    def __init__(*args, **kwargs):
        pass

    @trace
    def __call__(*args, **kwargs):
        # print(f"Trace from ABC__IgnoreTyping.call({args}, {kwargs})")
        return __ignore

    @trace
    def __getitem__(self, arg):
        return __ignore

    @trace
    def __getattr__(self, name):
        # print(f"Trace from ABC__IgnoreTyping.__getattr__({name}")
        if name in self.__dict__:
            return self.__dict__[name]
        return __ignore


class Shape(ABC):
    @abstractmethod
    def get_area(self) -> float:
        pass

    @abstractmethod
    def get_perimeter(self) -> float:
        pass


class Circle(Shape):
    def __init__(self, radius) -> None:
        self.radius = radius

    def get_area(self) -> float:
        return pi * self.radius**2

    def get_perimeter(self) -> float:
        return 2 * pi * self.radius


class Square(Shape):
    def __init__(self, side) -> None:
        self.side = side

    def get_area(self) -> float:
        return self.side**2

    def get_perimeter(self) -> float:
        return 4 * self.side


c1 = Circle(5)
s1 = Square(5)

for shape in [c1, s1]:
    a = shape.get_area()
    p = shape.get_perimeter()

    print(a, p)
    assert isinstance(a, (float, int)), "Area should be a float"
    assert isinstance(p, (float, int)), "Perimeter should be a float"

True
78.53982 31.41593
25 20


In [44]:
# %%micropython

from abc import ABCMeta


class MyABC(metaclass=ABCMeta):
    pass

MCUException: TypeError: function doesn't take keyword arguments


In [45]:
# %%micropython

from abc import get_cache_token, update_abstractmethods


class C(ABC):
    @abstractmethod
    def my_abstract_method(self, arg1): ...
    @classmethod
    @abstractmethod
    def my_abstract_classmethod(cls, arg2): ...
    @staticmethod
    @abstractmethod
    def my_abstract_staticmethod(arg3): ...

    @property
    @abstractmethod
    def my_abstract_property(self): ...
    @my_abstract_property.setter
    @abstractmethod
    def my_abstract_property(self, val): ...

    @abstractmethod
    def _get_x(self): ...
    @abstractmethod
    def _set_x(self, val): ...

    x = property(_get_x, _set_x)


token = get_cache_token()

cls = update_abstractmethods(C)

In [46]:
# %%micropython

from __future__ import annotations

from typing import cast
from typing_extensions import reveal_type

x = 1
reveal_type(x)
y = cast(str, x)
reveal_type(y)

try:
    y.upper()
except AttributeError as e:
    print("OK, Expected error:", e)

OK, Expected error: 'int' object has no attribute 'upper'


test that to validate the other typing related modules for runtime type checking in MicroPython.

In [47]:
# %%micropython

from __future__ import annotations

from typing import cast
from typing_extensions import reveal_type

x = 1
reveal_type(x)
y = cast(str, x)
reveal_type(y)
try:
    y.upper()
except AttributeError as e:
    # https://docs.python.org/3/library/typing.html#typing.cast
    print(f"OK - Intentional no runtime check: {e}")

OK - Intentional no runtime check: 'int' object has no attribute 'upper'


In [48]:
# %%micropython

from typing import Dict, ParamSpec, Union, get_origin


# https://docs.python.org/3/library/typing.html#typing.get_origin

assert get_origin(str) is None, "str"

# Partial implementation of get_origin

assert get_origin(Dict[str, int]) is dict, "origin Dict cannot be detected"
# assert get_origin(Union[int, str]) is Union, "Union"
# P = ParamSpec("P")
# assert get_origin(P.args) is P, "ParamSpec args"
# assert get_origin(P.kwargs) is P, "ParamSpec kwargs"

MCUException: AssertionError: origin Dict cannot be detected


In [49]:
# %%micropython

# ParamSpec, 3.11 notation
# https://docs.python.org/3/library/typing.html#typing.ParamSpec

from collections.abc import Callable
from typing import TypeVar, ParamSpec

T = TypeVar("T")
P = ParamSpec("P")


def add_logging(f: Callable[P, T]) -> Callable[P, T]:
    """A type-safe decorator to add logging to a function."""

    def inner(*args: P.args, **kwargs: P.kwargs) -> T:
        print(f"{f.__name__} was called")
        return f(*args, **kwargs)

    return inner


@add_logging
def add_two(x: float, y: float) -> float:
    """Add two numbers together."""
    return x + y


x = add_two(1, 2)
print(x)
assert x == 3, "add_two(1, 2) == 3"

add_two was called
3


In [50]:
# %%micropython

from typing import no_type_check


@no_type_check
def foo(x: int) -> str:
    return x


print(foo("42"))
assert foo(42) == 42

42


In [51]:
# %%micropython

from typing import overload


@overload
def bar(x: int) -> str: ...


@overload
def bar(x: str) -> int: ...


def bar(x):
    return x


print(bar(42))
assert bar(42) == 42

42


In [52]:
# %%micropython

from typing import NewType

UserId = NewType("UserId", int)
some_id = UserId(524313)

print(some_id)

assert isinstance(some_id, int)

524313


In [53]:
# %%micropython

import os

try:
    os.mkdir("collections")
except OSError as e:
    pass

validate typing of async typing in MicroPython.

In [54]:
# %%micropython
from collections.abc import Callable, Awaitable


def feeder(get_next_item: Callable[[], str]) -> None: ...  # Body


def async_query(
    on_success: Callable[[int], None], on_error: Callable[[int, Exception], None]
) -> None: ...  # Body


async def on_update(value: str) -> None: ...  # Body


callback: Callable[[str], Awaitable[None]] = on_update

# ...


def concat(x: str, y: str) -> str:
    return x + y


x: Callable[..., str]
x = str  # OK
x = concat  # Also OK


# ####

from collections.abc import Iterable
from typing import Protocol


class Combiner(Protocol):
    def __call__(self, *vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


def batch_proc(data: Iterable[bytes], cb_results: Combiner) -> bytes:
    for item in data:
        ...


def good_cb(*vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


batch_proc([], good_cb)  # OK

In [55]:
# %%micropython
from collections.abc import Callable, Awaitable

import asyncio


async def blink(led, period_ms):
    while True:
        await asyncio.sleep_ms(5)
        print(f"fake {led} ON")
        await asyncio.sleep_ms(period_ms)
        print(f"fake {led} OFF")


async def work(todo: List[Callable[[], Awaitable[None]]], timeout_ms: int) -> None:
    for task in todo:
        asyncio.create_task(task())
    await asyncio.sleep_ms(timeout_ms)


async def main():
    await work(
        [
            lambda: blink("LED1", 70),
            lambda: blink("led2", 20),
        ],
        90,
    )


try:
    asyncio.run(main())
finally:
    asyncio.new_event_loop()  # Clear retained stat

fake LED1 ON
fake led2 ON
fake led2 OFF
fake led2 ON
fake led2 OFF
fake led2 ON
fake LED1 OFF
fake led2 OFF
fake LED1 ON
fake led2 ON


In [56]:
# %%micropython
# https://docs.python.org/3/library/typing.html#generics

from collections.abc import Mapping, Sequence


class Employee: ...


# Sequence[Employee] indicates that all elements in the sequence
# must be instances of "Employee".
# Mapping[str, str] indicates that all keys and all values in the mapping
# must be strings.
def notify_by_email(employees: Sequence[Employee], overrides: Mapping[str, str]) -> None: ...

In [57]:
# %%micropython


from collections.abc import Mapping

# Type checker will infer that all elements in ``x`` are meant to be ints
x: list[int] = []

# Type checker error: ``list`` only accepts a single type argument:
y: list[int, str] = [1, "foo"]

# Type checker will infer that all keys in ``z`` are meant to be strings,
# and that all values in ``z`` are meant to be either strings or ints
z: Mapping[str, str | int] = {}

print(x, y, z)

[] [1, 'foo'] {}


In [58]:
# %%micropython

from typing import Generator


def echo_round() -> Generator[int, float, str]:
    sent = yield 0
    while sent >= 0:
        sent = yield round(sent)
    return "Done"


e = echo_round()

output = next(e)
print(output)
assert output == 0

0


In [59]:
# %%micropython

from typing import Any

a: Any = None
a = []  # OK
a = 2  # OK

s: str = ""
s = a  # OK


def foo(item: Any) -> int:
    # Passes type checking; 'item' could be any type,
    # and that type might have a 'bar' method
    item.bar()
    return 42


def hash_b(item: Any) -> int:
    try:
        # Passes type checking
        item.magic()
    except AttributeError:
        # just ignore any error for this test
        pass
    ...


# Passes type checking, since Any is compatible with all types
print(hash_b(42))
print(hash_b("foo"))

None
None


In [60]:
# %%micropython

from typing import AnyStr


def concat(a: AnyStr, b: AnyStr) -> AnyStr:
    return a + b


concat("foo", "bar")  # OK, output has type 'str'
concat(b"foo", b"bar")  # OK, output has type 'bytes'
try:
    concat("foo", b"bar")  # Error, cannot mix str and bytes
except TypeError as e:
    print("OK, expected:", e)

OK, expected: unsupported types for __add__: 'str', 'bytes'


In [61]:
# %%micropython

from typing import LiteralString


def run_query(sql: LiteralString) -> None: ...


def caller(arbitrary_string: str, literal_string: LiteralString) -> None:
    run_query("SELECT * FROM students")  # OK
    run_query(literal_string)  # OK
    run_query("SELECT * FROM " + literal_string)  # OK
    run_query(arbitrary_string)  # type checker error
    run_query(f"SELECT * FROM students WHERE name = {arbitrary_string}")  # type checker error

    assert isinstance(literal_string, str), "literal_string should be a string"
    assert isinstance(arbitrary_string, str), "arbitrary_string should be a string"


some_str = "a" * 1000
literal_str = "drop * from tables"

caller(some_str, literal_str)

In [62]:
# %%micropython

from typing import NoReturn


def hard_stop() -> NoReturn:
    raise RuntimeError("stop execution here")


print("hello")
hard_stop()

# this line will not be executed - but this is not shown a jupyter notebook
print("world")

hello


MCUException: RuntimeError: stop execution here


In [63]:
# %%micropython

from typing import Self, reveal_type


class Foo:
    def return_self(self) -> Self:
        ...
        return self


class SubclassOfFoo(Foo):
    pass


# reveal type does not work in micropython

print(reveal_type(Foo().return_self()))  # Revealed type is "Foo"
print(reveal_type(SubclassOfFoo().return_self()))  # Revealed type is "SubclassOfFoo"

None
None


typing_extensions

This is relevant to MicroPython as it is based on 3.4 / 3.5 syntax.

Enable use of new type system features on older Python versions. For example, typing.TypeGuard is new in Python 3.10, but typing_extensions allows users on previous Python versions to use it too.


see: https://typing-extensions.readthedocs.io/

In [73]:
# %%micropython

# In older versions of Python, TypeVarTuple and Unpack
# are located in the `typing_extensions` backports package.
from typing_extensions import TypeVarTuple, Unpack

Ts = TypeVarTuple("Ts")
tup: tuple[Unpack[Ts]]  # Semantically equivalent, and backwards-compatible

In [87]:
# %%micropython

from typing_extensions import TypeVar, reveal_type

Self = TypeVar("Self", bound="Foo")


class Foo:
    def return_self(self: Self) -> Self:
        ...
        return self


foo = Foo()
reveal_type(foo.return_self())  # Revealed type is "Foo"

Foo


In [86]:
# %%micropython
from typing_extensions import Self, reveal_type


class Foo:
    def return_self(self) -> Self:
        ...
        return self


class SubclassOfFoo(Foo):
    pass


print(reveal_type(Foo().return_self()))  # Revealed type is "Foo"
print(reveal_type(SubclassOfFoo().return_self()))  # Revealed type is "SubclassOfFoo"

Foo
<Foo object at 2000a050>
SubclassOfFoo
<SubclassOfFoo object at 2000a0c0>


In [124]:
# %%micropython

from typing import Generic, TypeVar


def trace(func):
    def wrapper(*args, **kwargs):
        print(f"Trace: {func.__name__} called with args={args}, kwargs={kwargs}")
        return func(*args, **kwargs)

    return wrapper


class Generic:
    # @trace
    def __init__(*args, **kwargs):
        pass

    # @trace
    def __getitem__(self, arg):
        return 1

    ...

    # @trace
    def __call__(*args, **kwargs):
        # May need some guardrails here
        pass

    # @trace
    def __getattr__(self, name):
        if name in self.__dict__:
            return self.__dict__[name]
        return __ignore


class TypeVar:
    @trace
    def __init__(self, name, *constraints, bound=None, covariant=False, contravariant=False):
        self.__name__ = name


T = TypeVar("T")

print("runtime -------------------")
print(repr(T))
print(Generic[T])


class Box(Generic[T]):
    def __init__(self, content: T) -> None:
        self.content = content

    def get_content(self) -> T:
        return self.content


print("runtime -------------------")
# Example usage
box_of_strings = Box[str]("Hello, World!")
print(box_of_strings.get_content())  # Output: Hello, World!

box_of_integers = Box[int](123)
print(box_of_integers.get_content())  # Output: 123

Trace: __init__ called with args=(<TypeVar object at 20008870>, 'T'), kwargs={}
runtime -------------------
<TypeVar object at 20008870>


MCUException: TypeError: 'type' object isn't subscriptable


In [ ]:
# %%micropython

from abc import ABC, abstractmethod
from math import pi


class Shape(ABC):
    @abstractmethod
    def get_area(self) -> float:
        pass

    @abstractmethod
    def get_perimeter(self) -> float:
        pass


class Circle(Shape):
    def __init__(self, radius) -> None:
        self.radius = radius

    def get_area(self) -> float:
        return pi * self.radius**2

    def get_perimeter(self) -> float:
        return 2 * pi * self.radius


class Square(Shape):
    def __init__(self, side) -> None:
        self.side = side

    def get_area(self) -> float:
        return self.side**2

    def get_perimeter(self) -> float:
        return 4 * self.side

In [ ]:
# %%micropython

from __future__ import annotations

from typing import cast
from typing_extensions import reveal_type

x = 1
reveal_type(x)
y = cast(str, x)
reveal_type(y)
try:
    y.upper()
except AttributeError as e:
    # https://docs.python.org/3/library/typing.html#typing.cast
    print(f"OK - Intentional no runtime check: {e}")

OK - Intentional no runtime check: 'int' object has no attribute 'upper'


In [ ]:
# %%micropython

from typing import Dict, ParamSpec, Union, get_origin


# https://docs.python.org/3/library/typing.html#typing.get_origin

assert get_origin(str) is None, "str"

# Partial implementation of get_origin

# assert get_origin(Dict[str, int]) is dict, "Dict"
# assert get_origin(Union[int, str]) is Union, "Union"
# P = ParamSpec("P")
# assert get_origin(P.args) is P, "ParamSpec args"
# assert get_origin(P.kwargs) is P, "ParamSpec kwargs"

In [ ]:
# %%micropython

# ParamSpec, 3.11 notation
# https://docs.python.org/3/library/typing.html#typing.ParamSpec

from collections.abc import Callable
from typing import TypeVar, ParamSpec

T = TypeVar("T")
P = ParamSpec("P")


def add_logging(f: Callable[P, T]) -> Callable[P, T]:
    """A type-safe decorator to add logging to a function."""

    def inner(*args: P.args, **kwargs: P.kwargs) -> T:
        print(f"{f.__name__} was called")
        return f(*args, **kwargs)

    return inner


@add_logging
def add_two(x: float, y: float) -> float:
    """Add two numbers together."""
    return x + y


x = add_two(1, 2)
print(x)

add_two was called
3


In [ ]:
# %%micropython
from typing import get_args

# partial implementation of get_args
assert get_args(int) == ()
# assert get_args(Dict[int, str]) == (int, str)
# assert get_args(Union[int, str]) == (int, str)

In [ ]:
# %%micropython

from typing import no_type_check


@no_type_check
def foo(x: int) -> str:
    return x


print(foo("42"))

42


In [ ]:
# %%micropython

from typing import overload


@overload
def bar(x: int) -> str: ...


@overload
def bar(x: str) -> int: ...


def bar(x):
    return x


print(bar(42))

42


In [ ]:
# %%micropython

from typing import NewType

UserId = NewType("UserId", int)
some_id = UserId(524313)

print(some_id)

assert isinstance(some_id, int)

524313


In [ ]:
# %%micropython

import os

try:
    os.mkdir("collections")
except OSError as e:
    pass

In [ ]:
# %%micropython
import collections

print(dir(collections))
print(collections.__dict__)

['__class__', '__name__', '__dict__', '__file__', '__path__', 'abc']
{'__path__': '/lib/collections', 'abc': <module 'collections.abc' from '/lib/collections/abc.mpy'>, '__name__': 'collections', '__file__': '/lib/collections/__init__.mpy'}


In [ ]:
# %%micropython
from collections.abc import Callable, Awaitable


def feeder(get_next_item: Callable[[], str]) -> None: ...  # Body


def async_query(
    on_success: Callable[[int], None], on_error: Callable[[int, Exception], None]
) -> None: ...  # Body


async def on_update(value: str) -> None: ...  # Body


callback: Callable[[str], Awaitable[None]] = on_update

# ...


def concat(x: str, y: str) -> str:
    return x + y


x: Callable[..., str]
x = str  # OK
x = concat  # Also OK


# ####

from collections.abc import Iterable
from typing import Protocol


class Combiner(Protocol):
    def __call__(self, *vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


def batch_proc(data: Iterable[bytes], cb_results: Combiner) -> bytes:
    for item in data:
        ...


def good_cb(*vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


batch_proc([], good_cb)  # OK

In [ ]:
# %%micropython
import os

In [ ]:
# %%micropython
# https://docs.python.org/3/library/typing.html#generics

from collections.abc import Mapping, Sequence


class Employee: ...


# Sequence[Employee] indicates that all elements in the sequence
# must be instances of "Employee".
# Mapping[str, str] indicates that all keys and all values in the mapping
# must be strings.
def notify_by_email(employees: Sequence[Employee], overrides: Mapping[str, str]) -> None: ...

In [ ]:
# %%micropython


from collections.abc import Mapping

# Type checker will infer that all elements in ``x`` are meant to be ints
x: list[int] = []

# Type checker error: ``list`` only accepts a single type argument:
y: list[int, str] = [1, "foo"]

# Type checker will infer that all keys in ``z`` are meant to be strings,
# and that all values in ``z`` are meant to be either strings or ints
z: Mapping[str, str | int] = {}

In [ ]:
# %%micropython

from typing import Generator


def echo_round() -> Generator[int, float, str]:
    sent = yield 0
    while sent >= 0:
        sent = yield round(sent)
    return "Done"


e = echo_round()
print(next(e))

0


In [ ]:
# %%micropython

from typing import Any

a: Any = None
a = []  # OK
a = 2  # OK

s: str = ""
s = a  # OK


def foo(item: Any) -> int:
    # Passes type checking; 'item' could be any type,
    # and that type might have a 'bar' method
    item.bar()
    return 42


def hash_b(item: Any) -> int:
    try:
        # Passes type checking
        item.magic()
    except AttributeError:
        # just ignore any error for this test
        pass
    ...


# Passes type checking, since Any is compatible with all types
hash_b(42)
hash_b("foo")

In [ ]:
# %%micropython

from typing import AnyStr


def concat(a: AnyStr, b: AnyStr) -> AnyStr:
    return a + b


concat("foo", "bar")  # OK, output has type 'str'
concat(b"foo", b"bar")  # OK, output has type 'bytes'
try:
    concat("foo", b"bar")  # Error, cannot mix str and bytes
except TypeError as e:
    print("OK, expected:", e)

OK, expected: unsupported types for __add__: 'str', 'bytes'


In [ ]:
# %%micropython

from typing import LiteralString


def run_query(sql: LiteralString) -> None: ...


def caller(arbitrary_string: str, literal_string: LiteralString) -> None:
    run_query("SELECT * FROM students")  # OK
    run_query(literal_string)  # OK
    run_query("SELECT * FROM " + literal_string)  # OK
    run_query(arbitrary_string)  # type checker error
    run_query(f"SELECT * FROM students WHERE name = {arbitrary_string}")  # type checker error


some_str = "a" * 1000
literal_str = "drop * from tables"

caller(some_str, literal_str)

In [ ]:
# %%micropython

from typing import NoReturn


def stop() -> NoReturn:
    raise RuntimeError("no way")

In [ ]:
# %%micropython

from typing import Self, reveal_type


class Foo:
    def return_self(self) -> Self:
        ...
        return self


class SubclassOfFoo(Foo):
    pass


print(reveal_type(Foo().return_self()))  # Revealed type is "Foo"
print(reveal_type(SubclassOfFoo().return_self()))  # Revealed type is "SubclassOfFoo"

None
None


In [ ]:
# %%micropython

# ABCs for working with IO
# Generic type IO[AnyStr] and its subclasses TextIO(IO[str]) and BinaryIO(IO[bytes]) represent the types of I/O streams such as returned by open().

from typing import IO
from typing import TextIO
from typing import BinaryIO

print("TODO: Add some tests")

TODO: Add some tests


In [ ]:
# %%micropython

from typing import runtime_checkable, Protocol


@runtime_checkable
class Closable(Protocol):
    def close(self): ...


try:
    assert isinstance(open("lib/typing.mpy"), Closable)
except TypeError as e:
    print(f"@runtime_checkable, not supported: {e}")

@runtime_checkable, not supported: issubclass() arg 2 must be a class or a tuple of classes


In [ ]:
# %%micropython

# 3.12 type parameter syntax
# https://docs.python.org/3/reference/simple_stmts.html#the-type-statement


# type Point = tuple[float, float]

print("OK, 3.12 syntax not supported")

OK, 3.12 syntax not supported


In [ ]:
# %%micropython

# 3.12 type parameter syntax not supported

# from collections.abc import Sequence

# def first[T](l: Sequence[T]) -> T:  # Function is generic over the TypeVar "T"
#     return l[0]

# from collections.abc import Sequence
# from typing import TypeVar

# U = TypeVar('U')                  # Declare type variable "U"

# def second(l: Sequence[U]) -> U:  # Function is generic over the TypeVar "U"

print("OK, 3.12 syntax not supported")

OK, 3.12 syntax not supported


In [ ]:
# # %%micropython

# from abc import ABCMeta


# class MyABC(metaclass=ABCMeta):
#     pass

print("metaclass not supported")

metaclass not supported


In [ ]:
from typing import final


class Base:
    @final
    def done(self) -> None: ...
class Sub(Base):
    def done(self) -> None:  # Error reported by type checker
        ...


@final
class Leaf: ...


class Other(Leaf):  # Error reported by type checker
    ...


other = Other()

True


MCUException: TypeError: can't create 'NoneType' instances


asyncio

In [64]:
# %%micropython

# ABCs for working with IO
# Generic type IO[AnyStr] and its subclasses TextIO(IO[str]) and BinaryIO(IO[bytes]) represent the types of I/O streams such as returned by open().

from typing import IO
from typing import TextIO
from typing import BinaryIO

print("TODO: Add some tests")

TODO: Add some tests


In [65]:
# %%micropython

from typing import runtime_checkable, Protocol


@runtime_checkable
class Closable(Protocol):
    def close(self): ...


try:
    assert isinstance(open("lib/typing.mpy"), Closable)
except TypeError as e:
    print(f"@runtime_checkable, not supported: {e}")

@runtime_checkable, not supported: issubclass() arg 2 must be a class or a tuple of classes


In [66]:
# %%micropython

# 3.12 type parameter syntax
# https://docs.python.org/3/reference/simple_stmts.html#the-type-statement


# type Point = tuple[float, float]

print("OK, 3.12 syntax not supported")

OK, 3.12 syntax not supported


In [67]:
# %%micropython

# 3.12 type parameter syntax not supported

# from collections.abc import Sequence

# def first[T](l: Sequence[T]) -> T:  # Function is generic over the TypeVar "T"
#     return l[0]

# from collections.abc import Sequence
# from typing import TypeVar

# U = TypeVar('U')                  # Declare type variable "U"

# def second(l: Sequence[U]) -> U:  # Function is generic over the TypeVar "U"

print("OK, 3.12 syntax not supported")

OK, 3.12 syntax not supported


In [68]:
# # %%micropython

# from abc import ABCMeta


# class MyABC(metaclass=ABCMeta):
#     pass

print("metaclass not supported")

metaclass not supported


In [69]:
# %%micropython

#
# Currently not implemented in micropython
#

# from typing import get_args

# partial implementation of get_args
# assert get_args(int) == ()
# assert get_args(Dict[int, str]) == (int, str)
# assert get_args(Union[int, str]) == (int, str)